In [ ]:
# add part emb
# add elapsed time
# saint model

In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb
import os
import pickle


debug = True
validaten_flg = False
EXP_ID = 'exp004'
CV_NUM = 'cv1'


def unpickle(filename):
    with open(filename, mode='rb') as fo:
        p = pickle.load(fo)
    return p


class CFG:
    INPUT_DIR = '../input/riiid-test-answer-prediction'
    OUT_DIR = '../input/riiid-trained-models-exp004'
    DATA_DIR = '../input/riiid-cv-datasets'
    TRAIN_PATH = os.path.join(INPUT_DIR, "train.csv")
    TEST_PATH = os.path.join(INPUT_DIR, "example_test.csv")
    LECTURE_PATH = os.path.join(INPUT_DIR, "lectures.csv")
    QUESTION_PATH = os.path.join(INPUT_DIR, "questions.csv")
    SUBMISSION_PATH = os.path.join(INPUT_DIR, "example_sample_submission.csv")
    SEED = 6718
    
    
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/riiid-test-answer-prediction/questions.csv'
debug = True
validaten_flg = False

In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import psutil
import os
import time
import sys
import math
from contextlib import contextmanager

@contextmanager
def trace(title):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    yield
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    print(f"[{m1:.1f}GB({sign}{delta:.1f}GB):{time.time() - t0:.1f}sec] {title} ", file=sys.stderr)
    

def seed_everything(seed=1129):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
    
seed_everything(CFG.SEED)

In [ ]:
with trace(''):
    # read data
    #feld_needed = ['timestamp', 'user_id', 'content_id', 'content_type_id', 'answered_correctly', 'prior_question_elapsed_time']
    #train = pd.read_pickle(train_pickle)[feld_needed] # [-10000000:]
    #valid = pd.read_pickle(valid_pickle)[feld_needed] # [-2500000:]


    #train = train.loc[train.content_type_id == False].reset_index(drop=True)
    #valid = valid.loc[valid.content_type_id == False].reset_index(drop=True)
    
    
    # part
    questions_df = pd.read_csv(question_file)[['question_id', 'part', 'tags', 'bundle_id']]

    #train = train.merge(questions_df[questions_df.question_id.isin(train.content_id)][['part']],
    #            how='left', left_on='content_id', right_index=True)

    #valid = valid.merge(questions_df[questions_df.question_id.isin(valid.content_id)][['part']],
    #            how='left', left_on='content_id', right_index=True)

In [ ]:
import pickle


def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)

def unpickle(filename):
    with open(filename, mode='rb') as fo:
        p = pickle.load(fo)
    return p  


with trace(''):
    group = unpickle('../input/riiid-transformer-all-v7-add-elapsed-time-pre/group.pkl')
    val_group = unpickle('../input/riiid-transformer-all-v7-add-elapsed-time-pre/val_group.pkl')
    skills = unpickle('../input/riiid-transformer-all-v7-add-elapsed-time-pre/skills.pkl')
    # lag_time_n_unq = unpickle('../input/riiid-transformer-all-v5-add-elapsed-time-pre/lag_time_n_unq.pkl')
    elapsed_time_n_unq = unpickle('../input/riiid-transformer-all-v7-add-elapsed-time-pre/elapsed_time_n_unq.pkl')

In [ ]:
MAX_SEQ = 160


with trace(''):
    # skills = train["content_id"].unique()
    # n_skill = len(skills)
    n_skill = 13523
    print("number skills", len(skills))

In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.samples = {}
        
        self.user_ids = []
        for user_id in group.index:
            q, qa, pt, elt = group[user_id]
            if len(q) < 5:
                continue
            
            # Main Contribution
            if len(q) > self.max_seq:
                total_questions = len(q)
                initial = total_questions % self.max_seq
                if initial >= 5:
                    self.user_ids.append(f"{user_id}_0")
                    self.samples[f"{user_id}_0"] = (q[:initial], qa[:initial], pt[:initial], elt[:initial])
                for seq in range(total_questions // self.max_seq):
                    self.user_ids.append(f"{user_id}_{seq+1}")
                    start = initial + seq * self.max_seq
                    end = start + self.max_seq
                    self.samples[f"{user_id}_{seq+1}"] = (q[start:end], qa[start:end], pt[start:end], elt[start:end])
            else:
                user_id = str(user_id)
                self.user_ids.append(user_id)
                self.samples[user_id] = (q, qa, pt, elt)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_, pt_, elt_ = self.samples[user_id]
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        pt = np.zeros(self.max_seq, dtype=int)
        elt = np.zeros(self.max_seq, dtype=int)
        
        if seq_len == self.max_seq:
            q[:] = q_
            qa[:] = qa_
            pt[:] = pt_
            elt[:] = elt_

        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
            pt[-seq_len:] = pt_
            elt[-seq_len:] = elt_

        
        target_id = q[1:]
        label = qa[1:]
        part_id = pt[1:]
        
        elapsed_time = elt[:-1]

        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill

        # x = qa[:-1].copy()
        return x, target_id, part_id, elapsed_time, label

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import copy


def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def get_mask(seq_len):
    ##todo add this to device
    return torch.from_numpy( np.triu(np.ones((seq_len ,seq_len)), k=1).astype('bool')).to(device)

def get_pos(seq_len):
    # use sine positional embeddinds
    return torch.arange( seq_len ).unsqueeze(0).to(device) 


class Feed_Forward_block(nn.Module):
    """
    out =  Relu( M_out*w1 + b1) *w2 + b2
    """
    def __init__(self, dim_ff):
        super().__init__()
        self.layer1 = nn.Linear(in_features=dim_ff , out_features=dim_ff)
        self.layer2 = nn.Linear(in_features=dim_ff , out_features=dim_ff)

    def forward(self,ffn_in):
        return  self.layer2(   F.relu( self.layer1(ffn_in) )   )
        

class Encoder_block(nn.Module):
    """
    M = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    O = SkipConct(FFN(LayerNorm(M)))
    """

    def __init__(self , dim_model, heads_en, total_ex ,total_pt, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.embd_ex =   nn.Embedding( total_ex , embedding_dim = dim_model )                   # embedings  q,k,v = E = exercise ID embedding, category embedding, and positionembedding.
        self.embd_pt =  nn.Embedding( total_pt, embedding_dim = dim_model )
        self.embd_pos   = nn.Embedding(  seq_len , embedding_dim = dim_model )                  #positional embedding

        self.multi_en = nn.MultiheadAttention( embed_dim= dim_model, num_heads= heads_en,  )     # multihead attention    ## todo add dropout, LayerNORM
        self.ffn_en = Feed_Forward_block( dim_model )                                            # feedforward block     ## todo dropout, LayerNorm
        self.layer_norm1 = nn.LayerNorm( dim_model )
        self.layer_norm2 = nn.LayerNorm( dim_model )


    def forward(self, in_ex, in_pt, first_block=True):

        ## todo create a positional encoding ( two options numeric, sine)
        if first_block:
            in_ex = self.embd_ex( in_ex )
            in_pt = self.embd_pt( in_pt )
            #in_pos = self.embd_pos( in_pos )
            #combining the embedings
            out = in_ex + in_pt #+ in_pos                      # (b,n,d)
        else:
            out = in_ex
        
        in_pos = get_pos(self.seq_len)
        in_pos = self.embd_pos( in_pos )
        out = out + in_pos                                      # Applying positional embedding

        out = out.permute(1,0,2)                                # (n,b,d)  # print('pre multi', out.shape )
        
        #Multihead attention                            
        n,_,_ = out.shape
        out = self.layer_norm1( out )                           # Layer norm
        skip_out = out 
        out, attn_wt = self.multi_en( out , out , out ,
                                attn_mask=get_mask(seq_len=n))  # attention mask upper triangular
        out = out + skip_out                                    # skip connection

        #feed forward
        out = out.permute(1,0,2)                                # (b,n,d)
        out = self.layer_norm2( out )                           # Layer norm 
        skip_out = out
        out = self.ffn_en( out )
        out = out + skip_out                                    # skip connection

        return out


class Decoder_block(nn.Module):
    """
    M1 = SkipConct(Multihead(LayerNorm(Qin;Kin;Vin)))
    M2 = SkipConct(Multihead(LayerNorm(M1;O;O)))
    L = SkipConct(FFN(LayerNorm(M2)))
    """

    def __init__(self, dim_model, total_in, total_et, heads_de, seq_len  ):
        super().__init__()
        self.seq_len    = seq_len
        self.embd_in    = nn.Embedding(total_in, embedding_dim = dim_model)                  #interaction embedding
        self.embd_et    = nn.Embedding(total_et, embedding_dim = dim_model)                  #elapsed time embedding
        self.embd_pos   = nn.Embedding(seq_len, embedding_dim = dim_model)                  #positional embedding
        self.multi_de1  = nn.MultiheadAttention(embed_dim=dim_model, num_heads=heads_de)  # M1 multihead for interaction embedding as q k v
        self.multi_de2  = nn.MultiheadAttention(embed_dim=dim_model, num_heads=heads_de)  # M2 multihead for M1 out, encoder out, encoder out as q k v
        self.ffn_en     = Feed_Forward_block(dim_model)                                         # feed forward layer

        self.layer_norm1 = nn.LayerNorm(dim_model)
        self.layer_norm2 = nn.LayerNorm(dim_model)
        self.layer_norm3 = nn.LayerNorm(dim_model)


    def forward(self, in_in, in_et, en_out, first_block=True):

         ## todo create a positional encoding ( two options numeric, sine)
        if first_block:
            in_in = self.embd_in(in_in)
            in_et = self.embd_et(in_et)

            #combining the embedings
            out = in_in + in_et #+ in_cat #+ in_pos                         # (b,n,d)
        else:
            out = in_in

        in_pos = get_pos(self.seq_len)
        in_pos = self.embd_pos( in_pos )
        out = out + in_pos                                          # Applying positional embedding

        out = out.permute(1,0,2)                                    # (n,b,d)# print('pre multi', out.shape )
        n,_,_ = out.shape

        #Multihead attention M1                                     ## todo verify if E to passed as q,k,v
        out = self.layer_norm1( out )
        skip_out = out
        out, attn_wt = self.multi_de1( out , out , out, 
                                     attn_mask=get_mask(seq_len=n)) # attention mask upper triangular
        out = skip_out + out                                        # skip connection

        #Multihead attention M2                                     ## todo verify if E to passed as q,k,v
        en_out = en_out.permute(1,0,2)                              # (b,n,d)-->(n,b,d)
        en_out = self.layer_norm2( en_out )
        skip_out = out
        out, attn_wt = self.multi_de2( out , en_out , en_out,
                                    attn_mask=get_mask(seq_len=n))  # attention mask upper triangular
        out = out + skip_out

        #feed forward
        out = out.permute(1,0,2)                                    # (b,n,d)
        out = self.layer_norm3( out )                               # Layer norm 
        skip_out = out
        out = self.ffn_en( out )                                    
        out = out + skip_out                                        # skip connection

        return out
    
    
class SAINT(nn.Module):
    def __init__(self, dim_model, num_en, num_de, heads_en, total_ex, total_pt, total_in, total_et, heads_de, seq_len):
        super().__init__( )

        self.num_en = num_en
        self.num_de = num_de

        self.encoder = get_clones(Encoder_block(dim_model, heads_en, total_ex, total_pt, seq_len), num_en)
        self.decoder = get_clones(Decoder_block(dim_model ,total_in, total_et, heads_de, seq_len), num_de)

        self.out = nn.Linear(in_features= dim_model, out_features=1)
    
    def forward(self, in_in, in_ex, in_pt, in_et):
        
        ## pass through each of the encoder blocks in sequence
        first_block = True
        for x in range(self.num_en):
            if x>=1:
                first_block = False
            in_ex = self.encoder[x](in_ex, in_pt, first_block=first_block)
            in_pt = in_ex                                  # passing same output as q,k,v to next encoder block

        
        ## pass through each decoder blocks in sequence
        first_block = True
        for x in range(self.num_de):
            if x>=1:
                first_block = False
            in_in = self.decoder[x](in_in, in_et, en_out=in_ex, first_block=first_block)

        ## Output layer
        # in_in = torch.sigmoid( self.out( in_in ) )
        in_in = self.out( in_in )
        return in_in.squeeze(-1)

In [ ]:
with trace(''):
    n_part = 7
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = SAINT(dim_model=128, 
                  num_en=3, 
                  num_de=3, 
                  heads_en=8, 
                  total_ex=n_skill+1, 
                  total_pt=n_part+1, 
                  total_in=2*n_skill+1, 
                  total_et=elapsed_time_n_unq+1, 
                  heads_de=8, 
                  seq_len=MAX_SEQ-1)
    
    model.load_state_dict(torch.load('../input/riiid-transformer-add-part-v11/SAKT-HDKIM.pt'))

    model.to(device)

In [ ]:
ADD = [
       'answered_correctly_true_weight_sum_u',
       'count_u_c',
       'count_c',
    
       'task_container_id', 
       'task_container_count',

       'answered_correctly_u_avg', 'elapsed_time_u_avg', 'explanation_u_avg',
       'answered_correctly_q_avg', 'elapsed_time_q_avg', 'explanation_q_avg', 
       'answered_correctly_uq_count', 'timestamp_u_recency_1', 'timestamp_u_recency_2', 'timestamp_u_recency_3', 
       'timestamp_u_incorrect_recency',
    
        'part_bundle_id', 
        'tags1', 'tags2', 'tags3', 
        'tags4', 'tags5', 'tags6', 
        'part_bundle_count',
        'count_u_tag1',
        'count_u_tag2',
        'count_u_tag3',
        'count_u_part_bundle_id',
        'task_container_id_u',
      ]

TARGET = 'answered_correctly'
FEATS = ['answered_correctly_avg_u', 'answered_correctly_sum_u', 'count_u', 'answered_correctly_avg_c', 'part', 'prior_question_had_explanation', 'prior_question_elapsed_time']
FEATS = FEATS + ADD

In [ ]:
"""
dro_cols = list(set(train.columns) - set(FEATS))
y_tr = train[TARGET]
y_va = valid[TARGET]
train.drop(dro_cols, axis=1, inplace=True)
valid.drop(dro_cols, axis=1, inplace=True)
_=gc.collect()
"""

In [ ]:
"""
lgb_train = lgb.Dataset(train[FEATS], y_tr)
lgb_valid = lgb.Dataset(valid[FEATS], y_va)
del train, y_tr
_=gc.collect()
"""

In [ ]:
"""
params = {'objective': 'binary', 
          'seed': CFG.SEED,
          # 'metric': 'auc',
          'num_leaves': 200,
          'feature_fraction': 0.75,
          'bagging_freq': 10,
          'bagging_fraction': 0.80
         }
    

model = lgb.train(
                    params=params,
                    train_set=lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    verbose_eval=100,
                    num_boost_round=10000,
                    early_stopping_rounds=10
                )
print('auc:', roc_auc_score(y_va, model.predict(valid[FEATS])))
_ = lgb.plot_importance(model)
"""

In [ ]:
# del lgb_train, lgb_valid; gc.collect()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=MAX_SEQ): #HDKIM 100
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills)
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]
        part_id = test_info["part"]
        # elapsed_time = test_info["prior_question_elapsed_time"]

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        pt = np.zeros(self.max_seq, dtype=int)
        elt = np.zeros(self.max_seq, dtype=int)
        
        if user_id in self.samples.index:
            q_, qa_, pt_, elt_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
                pt = pt_[-self.max_seq:]
                elt = elt_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_  
                pt[-seq_len:] = pt_    
                elt[-seq_len:] = elt_      
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        # x = qa[:-1].copy()
        
        questions = np.append(q[2:], [target_id])
        parts = np.append(pt[2:], [part_id])

        elapsed_times = elt[:-1]
        # elapsed_times = np.append(elt[2:], [elapsed_time])
        
        return x, questions, parts, elapsed_times

In [ ]:
import riiideducation
import psutil
model.eval()


env = riiideducation.make_env()
iter_test = env.iter_test()
set_predict = env.predict


previous_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if (previous_test_df is not None) & (psutil.virtual_memory().percent<90):
        print(psutil.virtual_memory().percent)
        previous_test_df[TARGET] = eval(test_df["prior_group_answers_correct"].iloc[0])
        previous_test_df = previous_test_df[previous_test_df.content_type_id == False]
        
        previous_test_df = previous_test_df.merge(questions_df[questions_df.question_id.isin(previous_test_df.content_id)][['part']],
                how='left', left_on='content_id', right_index=True)
    
        prev_group = previous_test_df[['user_id', 'content_id', 'answered_correctly', 'part', 'timestamp', 'prior_question_elapsed_time']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['part'].values,
            (r['prior_question_elapsed_time'].fillna(0).values/1000).astype(np.int64),
        ))
        
        for prev_user_id in prev_group.index:
            if prev_user_id in group.index:
                group[prev_user_id] = (
                    np.append(group[prev_user_id][0], prev_group[prev_user_id][0])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][1], prev_group[prev_user_id][1])[-MAX_SEQ:],
                    np.append(group[prev_user_id][2], prev_group[prev_user_id][2])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][3], prev_group[prev_user_id][3])[-MAX_SEQ:]
                )
 
            else:
                group[prev_user_id] = (
                    prev_group[prev_user_id][0], 
                    prev_group[prev_user_id][1],
                    prev_group[prev_user_id][2],
                    prev_group[prev_user_id][3]
                )
        
        # update_features_v4(previous_test_df, answered_correctly_sum_u_dict, count_u_dict,
        #                    pqhe_true_answered_correctly_sum_u_dict, count_c_dict,
        #                    answered_correctly_u_sum, answered_correctly_q_sum, timestamp_u_incorrect)
    
    
    previous_test_df = test_df.copy()
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df = test_df.merge(questions_df[questions_df.question_id.isin(test_df.content_id)][['part']],
                how='left', left_on='content_id', right_index=True)
    
    test_dataset = TestDataset(group, test_df, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in tqdm(test_dataloader):
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        part_id = item[2].to(device).long()
        elapsed_time = item[3].to(device).long()

        with torch.no_grad():
            output = model(x, target_id, part_id, elapsed_time)
        
        
        output = torch.sigmoid(output)
        output = output[:, -1]
        
        outs.extend(output.view(-1).data.cpu().numpy())
        
    
    """
    test_df = test_df.merge(questions_df[questions_df.question_id.isin(test_df.content_id)],
                how='left', left_on='content_id', right_index=True)
    test_df = test_df.merge(part_bundle_df[part_bundle_df.part_bundle_id.isin(test_df.part_bundle_id)],
                        how='left', on='part_bundle_id')
    test_df = test_df.merge(task_container_df[task_container_df.task_container_id.isin(test_df.task_container_id)],
                        how='left', on='task_container_id')
    test_df = test_df.merge(content_df[content_df.content_id.isin(test_df.content_id)],
                        how='left', on='content_id')
    
    test_df['prior_question_had_explanation'] = test_df.prior_question_had_explanation.fillna(False).astype('int8')
    test_df['prior_question_elapsed_time_mean'] = test_df.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
    
    test_df[TARGET] = 0
    test_df = add_user_feats_v4(test_df, count_u_tag1_dict, count_u_tag2_dict, count_u_tag3_dict,
                                count_u_part_bundle_id_dict, task_container_id_u_dict,
                                answered_correctly_sum_u_dict, count_u_dict,
                                pqhe_true_answered_correctly_sum_u_dict,
                                count_c_dict, count_u_c_dict,
                                answered_correctly_u_count, answered_correctly_u_sum, 
                                elapsed_time_u_sum, explanation_u_sum, timestamp_u, timestamp_u_incorrect, 
                                answered_correctly_q_count, answered_correctly_q_sum, elapsed_time_q_sum, explanation_q_sum, 
                                answered_correctly_uq, update = False)
    
    test_df[TARGET] =  model.predict(test_df[FEATS])   
    """
    
    test_df[TARGET] = outs
    
    set_predict(test_df[['row_id', TARGET]])

In [ ]:
# test_df[TARGET]

Have a fun with loops! :)